<a href="https://colab.research.google.com/github/ckjen168/LLMColab/blob/main/stk_ch03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH-03 股市資料蒐集、爬蟲與搭建資料庫

## 3-2 資料爬蟲

## 證交所資料爬蟲
1. 進入證交所網址：https://www.twse.com.tw/zh/index.html
2. 使用開發者模式取得請求資料網址

### 1️⃣ 匯入套件

In [ ]:
import requests
import pandas as pd
import datetime as dt # 時間套件
from dateutil.relativedelta import relativedelta

### 2️⃣ 取得個股日成交資訊

In [ ]:
# 輸入股票代號
stock_id = '2330'
# 當日時間
date = dt.date.today().strftime("%Y%m%d")
# 取得證交所網站資料
stock_data = requests.get(f'https://www.twse.com.tw/rwd/zh/ \
            afterTrading/STOCK_DAY?date={date}&stockNo={stock_id}')
json_data = stock_data.json()
df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
df.tail()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
5,113/07/08,"45,678,332","47,210,175,550","1,005.00","1,050.00","1,000.00","1,035.00",+30.00,"74,909"
6,113/07/09,"54,339,957","56,382,512,235","1,030.00","1,055.00","1,025.00","1,040.00",+5.00,"74,954"
7,113/07/10,"51,810,372","53,308,027,550","1,020.00","1,050.00","1,015.00","1,045.00",+5.00,"61,199"
8,113/07/11,"49,304,453","52,782,475,514","1,065.00","1,080.00","1,055.00","1,080.00",+35.00,"65,005"
9,113/07/12,"79,472,761","82,222,185,037","1,030.00","1,045.00","1,025.00","1,040.00",-40.00,"175,230"


### 3️⃣ 取得連續月份資料
以個股本益比為例

In [ ]:
# 設定抓取幾個月資料
month_num=3
date_now = dt.datetime.now()

# 建立日期串列
date_list = [(date_now - relativedelta(months=i)).replace(day=1).\
             strftime('%Y%m%d') for i in range(month_num)]

date_list.reverse()
all_df = pd.DataFrame()

# 使用迴圈抓取連續月份資料
for date in date_list:
  url = f'https://www.twse.com.tw/rwd/zh/afterTrading/\
      BWIBBU?date={date}&stockNo={stock_id}'
  try:
    json_data = requests.get(url).json()
    df = pd.DataFrame(data=json_data['data'],
                  columns=json_data['fields'])
    all_df = pd.concat([all_df, df], ignore_index=True)
  except Exception as e:
    print(f"無法取得{date}的資料, 可能資料量不足.")

all_df.head()

,日期,殖利率(%),股利年度,本益比,股價淨值比,財報年/季
0,113年05月02日,1.68,112,23.88,5.79,112/4
1,113年05月03日,1.67,112,24.13,5.85,112/4
2,113年05月06日,1.65,112,24.31,5.89,112/4
3,113年05月07日,1.63,112,24.74,6.00,112/4
4,113年05月08日,1.62,112,24.81,6.01,112/4


## 用 BeautifulSoup4 取得 Yahoo 股市資料

###4️⃣ 匯入套件

In [ ]:
from datetime import datetime
from bs4 import BeautifulSoup
import time

###  5️⃣ 取得當日股價

In [ ]:
def yahoo_stock(stock_id):
    url = f'https://tw.stock.yahoo.com/quote/{stock_id}.TW'
    # 使用 requests 取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用 Beautiful Soup 解析 HTML 內容
    soup = BeautifulSoup(html, 'html.parser')
    # 使用 find 與 find_all 定位元素
    time_element = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('time')
    table_soups = soup.find('section',\
                {'id': 'qsp-overview-realtime-info'}).find('ul')\
                                   .find_all('li')
    fields = []
    datas = []
    for table_soup in table_soups:
        table_datas = table_soup.find_all('span')
        for num,table_data in enumerate(table_datas):
            if table_data.text =='':
                continue
            if num == 0:
                fields.append(table_data.text)
            else:
                datas.append(table_data.text)
    # 建立 DataFrame
    df = pd.DataFrame([datas], columns=fields)
    # 增加日期和股號欄位
    df.insert(0,'日期',time_element['datatime'])
    df.insert(1,'股號',stock_id)
    # 回傳 DataFrame
    return df

yahoo_stock(stock_id)

,日期,股號,成交,開盤,最高,最低,均價,成交金額(億),昨收,漲跌幅,漲跌,總量,昨量,振幅
0,2023/12/18 11:48,2330,581,579,582,577,579,84.12,585,0.68%,4.00,"14,509","54,288",0.00%


### 6️⃣ 取得季報表資訊


In [ ]:
# 函式可用於奇摩財報
def url_find(url):
    words = url.split('/')
    k = words[-1]
    # 使用requests取得網頁內容
    response = requests.get(url)
    html = response.content
    # 使用Beautiful Soup解析HTML內容
    soup = BeautifulSoup(html, 'html.parser')
    # 找到表格的表頭
    table_soup = soup.find('section', {'id': 'qsp-{}-table'.format(k)})
    table_fields=table_soup.find('div', class_='table-header')
    table_fields_lines = list(table_fields.stripped_strings)
    # 找到數據
    data_rows = table_soup.find_all('li' ,class_='List(n)')
    # 解析資料行內容
    data = []
    for row in data_rows:
        row_data = list(row.stripped_strings)
        data.append(row_data)
    # 建立 DataFrame
    df = pd.DataFrame(data, columns=table_fields_lines)
    return df

# 抓損益表
url = f'https://tw.stock.yahoo.com/quote/{stock_id}/income-statement'
# 抓資產負債表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/balance-sheet'
# 抓現金流量表
# url = f'https://tw.stock.yahoo.com/quote/{stock_id}/cash-flow-statement'

# 抓取季報表資料
df = url_find(url).transpose()

# 資料處理
df.columns = df.iloc[0]
df = df[1:]
df.insert(0,'年度/季別',df.index)
df.columns.name = None
df.reset_index(drop=True, inplace=True)

df.tail()

,年度/季別,營業收入,營業毛利,營業費用,營業利益,稅後淨利
15,2019 Q4,"317,237,065","159,240,985","34,942,621","124,243,722","116,078,194"
16,2019 Q3,"293,045,439","139,432,161","31,378,953","107,887,292","101,102,454"
17,2019 Q2,"240,998,475","103,673,230","27,164,995","76,304,053","66,775,851"
18,2019 Q1,"218,704,469","90,352,125","26,018,013","64,266,023","61,387,310"
19,2018 Q4,"289,770,193","138,042,400","30,852,310","107,123,251","100,005,385"


##使用 selenium 做新聞爬蟲

### 7️⃣ 用 requests 取得股票新聞

In [ ]:
# 股票代號
stock_id = "2330"
# 預設表格數據和欄位
field=['股號','日期','標題','內容']
data=[]
# 取得 Json 格式資料
json_data = requests.get(f'https://ess.api.cnyes.com/ess/api/'
                f'v1/news/keyword?q={stock_id}&limit=20&page=1').json()
# 依照格式擷取資料
items=json_data['data']['items']
for item in items:
    # 網址、標題和日期
    news_id = item["newsId"]
    title = item["title"]
    publish_at = item["publishAt"]
    # 使用 UTC 時間格式
    utc_time = datetime.utcfromtimestamp(publish_at)
    formatted_date = utc_time.strftime('%Y-%m-%d')
    # 前往網址擷取內容
    url = requests.get(f'https://news.cnyes.com/'
                       f'news/id/{news_id}').content
    soup = BeautifulSoup(url, 'html.parser')
    p_elements = soup.find_all('p')
    # 提取段落内容
    p=''
    for paragraph in p_elements[4:]:
        p+=paragraph.get_text()
    data.append([stock_id, formatted_date ,title,p])
# 建立表格
df = pd.DataFrame(data,columns=field)
df

,股號,日期,標題,內容
0,2330,2023-12-15,"《異動股》中國上城(0<mark>2330</mark>)升逾41%,現報0.199元","《經濟通通訊社15日專訊》中國上城(02330)升41.1%,報0.199元,最高價0...."
1,2330,2023-12-15,台積電:WaferTech歡慶更名為TSMC Washington,第51款1.事實發生日:112/12/152.公司名稱:台灣積體電路製造股份有限公司3.與公...
2,2330,2023-12-14,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,第20款1.證券名稱:政府機構不動產抵押貸款債券。2.交易日期:112/11/1~112/1...
3,2330,2023-12-13,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,第20款1.證券名稱:公司債。2.交易日期:112/12/8~112/12/133.交易數量...
4,2330,2023-12-13,盤後速報 - 台積電(<mark>2330</mark>)次交易(14)日除息3元，參考價5...,台積電(2330-TW)次交易(14)日將進行除息交易，其中每股配發現金股利3元。首日參考價...
5,2330,2023-12-11,台積電:本公司代子公司 TSMC Global Ltd. 公告取得固定收益證券,第20款1.證券名稱:公司債。2.交易日期:112/12/7~112/12/113.交易數量...
6,2330,2023-12-08,台積電:台積公司2023年11月營收報告,第51款1.事實發生日:112/12/082.公司名稱:台灣積體電路製造股份有限公司3.與公...
7,2330,2023-12-07,盤後速報 - 台積電(<mark>2330</mark>)下週(12月14日)除息3元，預估...,台積電(2330-TW)下週(12月14日)將進行除息交易，其中每股配發現金股利3元。以今日...
8,2330,2023-12-06,台積電:本公司代重要子公司TSMC North America公告股東常會重要決議,第18款1.股東常會日期:112/12/062.重要決議事項一、盈餘分配或盈虧撥補:不適用。...
9,2330,2023-12-05,"中國上城(0<mark>2330</mark>)履行所有復牌指引,申請今早復牌","《經濟通通訊社6日專訊》停牌18個月的中國上城(02330)公布,履行所有復牌指引,已向..."


### 8️⃣  安裝及匯入套件

In [ ]:
!pip install selenium
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 不顯示瀏覽器
chrome_options.add_argument('--no-sandbox')# 以最高權限運行

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00


### 9️⃣ 使用 Selenium 取得股票新聞

In [ ]:
# 透過 options 設定 driver
driver = webdriver.Chrome(options=chrome_options)
data2=[] # 表格數據
# 目標網址
url = f"https://www.cnyes.com/search/news?keyword={stock_id}"
driver.get(url)

# 模擬滑動滑鼠滾輪的行為，用於加載更多內容
scroll_pause_time = 2  # 等待時間
last_height = driver.execute_script(
                    "return document.body.scrollHeight")
while True:
    driver.execute_script(
        "window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script(
        "return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

elements = driver.find_elements("xpath",
                        '//*[@id="_SearchAll"]/section/div/a')

# 擷取網址和標題
for element in elements:
    link = element.get_attribute("href")
    title = element.text
    title=title.split('\n')
    data2.append([stock_id, title[1] ,title[0],link])
# 關閉瀏覽器
driver.quit()
for link in data2:
  # 使用 requests 前往網址擷取新聞內容
  link_a = requests.get(link[3]).content
  link_b = BeautifulSoup(link_a,'html.parser')
  p_elements = link_b.find('article')
  # 取得段落内容
  try:
    link[3] = p_elements.get_text()
  except:
    link[3] = '無內容'
# 建立表格
df = pd.DataFrame(data2,columns=field)
df.tail()

,股號,日期,標題,內容
563,2330,2021/12/29,富驛-KY引新奧援 本土建商成最大股東後市具想像空間,連鎖商旅業者富驛 - KY(2724-TW) 今 (29) 日指出，今年立信機構透過二次增資...
564,2330,2021/12/29,2022年汽車智慧化 智慧座艙將成主戰場,文．高適\n汽車朝向智慧化發展，自動駕駛及智慧座艙是兩大主要演進方向。其中，智慧座艙被視為是...
565,2330,2021/12/24,外資連四買敲進晶圓雙雄逾2.3萬張 投信終止連八買,台股今 (24) 日盤中最高站上 18039.85 點，改寫歷史新高，但午盤過後電子權值、航...
566,2330,2021/12/24,綠電:更正本公司110年12月16日之公告本公司最近一年累積處分有價證券達公告標準(交易總金...,第20款公司代號：8440公司名稱：綠電發言日期：2021/12/24發言時間：16:42:...
567,2330,2021/12/24,台積電:本公司代子公司 TSMC Global Ltd. 公告處分固定收益證券,第20款公司代號：2330公司名稱：台積電發言日期：2021/12/24發言時間：15:24...


```
#把資料存成 csv 可以用以下這段
df.to_csv('/content/news_data.csv' )
```

#3.3 用 Python 套件輕鬆取得股市資料

##使用 yfinance 下載股市資料

### 🔟  安裝及匯入套件

In [ ]:
!pip install yfinance
import yfinance as yf

### 1️⃣1️⃣  設定股票代碼和起止時間

In [ ]:
# 指定要下載的股票代碼, 上市為 .TW;上櫃為 .TWO
stock_id = '2330.TW'
# 設定開始與結束時間
end = dt.date.today()
start = end - dt.timedelta(days=360)

### 1️⃣2️⃣ 取得每日股價 (開高低收) 資料

In [ ]:
stock_data = yf.download(stock_id, start=start, auto_adjust=False, multi_level_index=False)
stock_data.tail()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-04-16,802.0,803.0,785.0,788.0,788.0,49508969
2024-04-17,798.0,808.0,793.0,804.0,804.0,34472074
2024-04-18,796.0,810.0,792.0,804.0,804.0,43598591
2024-04-19,769.0,770.0,746.0,750.0,750.0,130968654
2024-04-22,740.0,757.0,740.0,742.0,742.0,46540798




```
# 依照資料期間下載
stock_data = yf.download(stock_id, period="3mo")

# 下載不同時間頻率的資料 (1分K)
stock_data = yf.download(stock_id, interval="1m")
```



### 1️⃣3️⃣ 取得多檔股票的資料

In [ ]:
stocks = [stock_id, '2303.TW', '2454.TW'] #分別為台積電、聯電和聯發科
stock_data = yf.download(stocks, start=start, end=end)
stock_data.tail()

[*********************100%%**********************]  3 of 3 completed


Adj Close                          Close                  \
              2303.TW     2330.TW 2454.TW    2303.TW 2330.TW 2454.TW   
Date                                                                   
2023-12-11  49.049999  571.015564   941.0  49.049999   574.0   941.0   
2023-12-12  49.900002  574.994812   939.0  49.900002   578.0   939.0   
2023-12-13  50.200001  574.000000   955.0  50.200001   577.0   955.0   
2023-12-14  50.400002  582.000000   966.0  50.400002   582.0   966.0   
2023-12-15  50.700001  585.000000   996.0  50.700001   585.0   996.0   

                 High                        Low                       Open  \
              2303.TW 2330.TW 2454.TW    2303.TW 2330.TW 2454.TW    2303.TW   
Date                                                                          
2023-12-11  49.299999   575.0   944.0  48.650002   570.0   932.0  49.000000   
2023-12-12  49.900002   581.0   950.0  49.200001   575.0   935.0  49.450001   
2023-12-13  50.200001   579.0   955.0  49.500000   576.0   939.0  50.000000   
2023-12-14  50.500000   582.0   969.0  50.000000   579.0   953.0  50.400002   
2023-12-15  51.000000   586.0   998.0  50.400002   580.0   971.0  50.700001   

                              Volume                      
           2330.TW 2454.TW   2303.TW   2330.TW   2454.TW  
Date                                                      
2023-12-11   572.0   942.0  28661262  27378610   5648215  
2023-12-12   580.0   950.0  48600982  29403010   4510280  
2023-12-13   576.0   944.0  47899791  22977731   5889291  
2023-12-14   581.0   961.0  51468459  39409958   6610988  
2023-12-15   585.0   975.0  83947122  54323262  11839614

### 1️⃣4️⃣ 取得公司基本資料


In [ ]:
stock = yf.Ticker(stock_id)
stock.info # 為字典型態

{'address1': 'Hsinchu Science Park',
 'address2': 'No. 8, Li-Hsin Road 6',
 'city': 'Hsinchu City',
 'zip': '300096',
 'country': 'Taiwan',
 'phone': '886 3 563 6688',
 'fax': '886 3 563 7000',
 'website': 'https://www.tsmc.com',
 'industry': 'Semiconductors',
 'industryKey': 'semiconductors',
 'industryDisp': 'Semiconductors',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Taiwan Semiconductor Manufacturing Company Limited, together with its subsidiaries, manufactures, packages, tests, and sells integrated circuits and other semiconductor devices in Taiwan, China, Europe, the Middle East, Africa, Japan, the United States, and internationally. It provides a range of wafer fabrication processes, including processes to manufacture complementary metal- oxide-semiconductor (CMOS) logic, mixed-signal, radio frequency, embedded memory, bipolar CMOS mixed-signal, and others. The company also offers customer and engineering support se

### 1️⃣5️⃣  取得損益表

In [ ]:
financials = stock.financials
financials.tail()

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Other Gand A,60872800000.0,53524800000.0,36929600000.0,28457600000.0
Gross Profit,1175110600000.0,1348354800000.0,819537300000.0,711130100000.0
Cost Of Revenue,986625200000.0,915536500000.0,767877700000.0,628124700000.0
Total Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0
Operating Revenue,2161735800000.0,2263891300000.0,1587415000000.0,1339254800000.0


In [ ]:
bs = stock.balance_sheet
bs

,2023-12-31,2022-12-31,2021-12-31,2020-12-31
Treasury Shares Number,0.0,NaN,NaN,NaN
Ordinary Shares Number,25932070992.0,25930380458.0,25930380458.0,25930380458.0
Share Issued,25932070992.0,25930380458.0,25930380458.0,25930380458.0
Total Debt,956257900000.0,888174400000.0,753631900000.0,367792300000.0
Tangible Book Value,3406755700000.0,2877020500000.0,2122438100000.0,1809043200000.0
...,...,...,...,...
Cash Cash Equivalents And Short Term Investments,1714803200000.0,1586500100000.0,1204913700000.0,799893200000.0
Other Short Term Investments,249375400000.0,243686000000.0,139923500000.0,139722600000.0
Cash And Cash Equivalents,1465427800000.0,1342814100000.0,1064990200000.0,660170600000.0
Cash Equivalents,12326200000.0,13522700000.0,6182100000.0,6590000000.0


### 1️⃣6️⃣ 取得法人持股比例
因 yfinance 資料問題，此儲存格暫時無法使用

In [ ]:
institutional_holders = stock.institutional_holders
institutional_holders.tail()

,Holder,Shares,Date Reported,% Out,Value
5,Capital World Growth and Income Fund,133697045,2023-09-30,0.0052,77811680190
6,Invesco Developing Markets Fund,112490429,2023-07-31,0.0043,65469429678
7,Fidelity Series Emerging Markets Opportunities...,94966577,2023-09-30,0.0037,55270547814
8,Growth Fund Of America Inc,81325000,2023-09-30,0.0031,47331150000
9,Vanguard International Growth Fund,74626000,2023-08-31,0.0029,43432332000


## 使用 FinMind 下載股市資料

### 1️⃣7️⃣  安裝及匯入套件

In [ ]:
!pip install FinMind
from FinMind.data import DataLoader
import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 761.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 13.4 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.5.25-py3-none-any.whl size=24861 sha256=ee0d659b7798e434075260816a926449ec7c69e93603d3f975daf056a0e3c2db
  Stored in directory: /root/.cache/pip/wheels/d4/bd/c4/0dda911b5461ee856352cd8c9e16472229bb51c95e74b74c08
Successfully built ta


### 1️⃣8️⃣  輸入 FinMind API 和帳號密碼

In [ ]:
token = getpass.getpass("請輸入 FinMind 金鑰：")

請輸入 FinMind 金鑰：··········


### 1️⃣9️⃣  建立 FinMind 資料庫物件和登入 FinMind

In [ ]:
api = DataLoader()
api.login_by_token(api_token=token)

### 2️⃣0️⃣ 取得股價資料

In [ ]:
# 股票代號
stock_id = '2330'
# 資料期間
end = dt.date.today()
start = end - dt.timedelta(days=360)

stock_data =  api.taiwan_stock_daily(
    stock_id=stock_id,
    start_date=start,
    end_date=end)

stock_data.tail()

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
230,2023-10-31,2330,28073280,14873552534,535.0,535.0,527.0,529.0,-3.0,32067
231,2023-11-01,2330,14028881,7432561094,533.0,533.0,527.0,528.0,-1.0,15452
232,2023-11-02,2330,31075778,16846430251,536.0,547.0,535.0,547.0,19.0,32075
233,2023-11-03,2330,16673456,9129495705,547.0,549.0,545.0,549.0,2.0,17911
234,2023-11-06,2330,31297186,17321453879,553.0,556.0,550.0,550.0,1.0,32684


### 2️⃣1️⃣ 取得損益表資料

In [ ]:
financial_data = api.taiwan_stock_financial_statement(
    stock_id=stock_id,
    start_date=str(start),)

financial_data.tail()

,date,stock_id,type,value,origin_name
43,2023-06-30,2330,TotalNonoperatingIncomeAndExpense,1.271687e+10,營業外收入及支出
44,2023-06-30,2330,OtherComprehensiveIncome,5.659104e+09,其他綜合損益（淨額）
45,2023-06-30,2330,GrossProfit,2.601998e+11,營業毛利（毛損）淨額
46,2023-06-30,2330,EquityAttributableToOwnersOfParent,1.817990e+11,淨利（淨損）歸屬於母公司業主
47,2023-06-30,2330,NoncontrollingInterests,-8.201500e+07,淨利（淨損）歸屬於非控制權益


### 2️⃣2️⃣ 取得法人買賣資料

In [ ]:
investors_data = api.taiwan_stock_institutional_investors(
    stock_id=stock_id,
    start_date=str(start),)
investors_data.tail()

,date,stock_id,buy,name,sell
1170,2023-11-06,2330,0,Foreign_Dealer_Self,0
1171,2023-11-06,2330,468000,Investment_Trust,572432
1172,2023-11-06,2330,173000,Dealer_self,181000
1173,2023-11-06,2330,224599,Dealer_Hedging,53700
1174,2023-11-06,2330,24502766,Foreign_Investor,11671452


## 使用 FinLab 下載股市資料
注意！FinLab 目前需付費才能取得最新資料

### 2️⃣3️⃣ 安裝及匯入套件

In [ ]:
!pip install finlab
import finlab
from finlab import data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
Mounted at /content/drive


### 2️⃣4️⃣ 登入 FinLab

In [ ]:
token = getpass.getpass("請輸入FinLab金鑰：")
finlab.login(token)

請輸入FinLab金鑰：··········
輸入成功!


### 2️⃣5️⃣ 取得收盤價

In [ ]:
close = data.get('price:收盤價')
close.tail()

Due to your status as a free user, the most recent data has been shortened or limited.
Daily usage: 12.2 / 500 MB - price:收盤價


symbol,0015,0050,0051,0052,0053,0054,0055,0056,0057,0058,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2020-12-25,NaN,118.95,42.83,106.50,55.85,27.57,17.69,29.51,78.55,NaN,...,22.20,40.60,16.10,11.9,10.05,88.5,19.95,112.5,NaN,9.73
2020-12-28,NaN,120.00,43.83,107.80,56.85,27.90,17.71,29.75,79.10,NaN,...,22.20,40.55,16.05,11.9,10.00,88.1,20.25,110.0,31.00,9.70
2020-12-29,NaN,119.90,43.79,108.00,56.80,27.90,17.71,29.67,79.60,NaN,...,22.25,40.60,16.05,11.9,9.97,87.8,19.85,109.0,34.10,9.70
2020-12-30,NaN,121.60,43.93,109.85,57.10,28.10,18.06,29.78,80.20,NaN,...,22.35,40.75,16.15,11.9,9.99,86.4,19.85,109.0,33.95,9.98
2020-12-31,NaN,122.25,44.00,110.20,57.70,28.21,18.05,29.95,80.40,NaN,...,22.50,40.90,16.15,11.9,10.15,91.3,19.75,107.0,34.00,9.98


### 2️⃣6️⃣ 選擇產業

In [ ]:
data.set_universe(market='TSE', category='半導體')
close = data.get('price:收盤價')
close.tail()

Daily usage: 12.3 / 500 MB - security_categories


symbol,2302,2303,2311,2325,2329,2330,2337,2338,2340,2342,...,6573,6756,8016,8028,8081,8110,8131,8150,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2020-12-25,24.50,46.75,NaN,NaN,14.90,511.0,40.40,40.35,25.4,36.95,...,14.95,262.0,159.0,56.9,158.5,14.05,37.45,34.40,53.0,39.35
2020-12-28,24.70,48.60,NaN,NaN,15.95,515.0,43.80,42.70,26.1,40.60,...,15.25,250.0,164.0,58.5,161.5,14.35,37.75,34.50,52.4,39.90
2020-12-29,23.80,47.40,NaN,NaN,15.50,515.0,41.85,41.50,25.9,39.50,...,15.10,236.0,161.0,58.2,159.5,13.90,37.70,35.00,51.3,39.45
2020-12-30,23.95,48.30,NaN,NaN,15.25,525.0,42.35,41.35,26.1,39.35,...,15.05,246.0,161.5,57.4,160.0,14.00,37.80,34.70,51.6,39.50
2020-12-31,24.50,47.15,NaN,NaN,15.05,530.0,42.30,40.35,27.5,38.65,...,15.10,238.0,162.5,58.7,159.5,13.80,37.80,34.35,52.9,39.30


### 2️⃣7️⃣ 取得財報資料

In [ ]:
df = data.get('financial_statement:每股盈餘')
df.tail()

Daily usage: 13.5 / 500 MB - financial_statement:每股盈餘


symbol,2302,2303,2311,2325,2329,2330,2337,2338,2340,2342,...,6789,6799,8016,8028,8081,8110,8131,8150,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2019-Q3,0.02,0.25,NaN,NaN,0.45,3.90,1.01,0.66,0.50,-1.10,...,NaN,NaN,3.22,0.83,2.50,0.41,0.80,0.81,0.22,1.34
2019-Q4,-0.08,0.32,1.41,1.46,0.08,4.48,0.41,0.62,0.38,-0.89,...,2.11,1.13,2.54,0.44,2.19,0.32,0.69,0.72,0.11,0.99
2020-Q1,-0.09,0.19,NaN,NaN,0.02,4.51,0.67,-1.09,0.32,-0.69,...,NaN,NaN,2.43,0.28,2.73,-0.27,0.88,0.98,0.29,1.53
2020-Q2,0.01,0.55,1.23,1.55,-0.50,4.66,0.72,0.66,0.19,-0.91,...,3.86,1.35,2.43,0.31,2.73,0.18,0.78,0.75,0.57,0.57
2020-Q3,0.21,0.75,NaN,NaN,0.02,5.30,0.88,1.27,0.55,2.00,...,NaN,NaN,2.63,0.11,3.55,0.42,0.80,0.58,0.68,0.24


### 2️⃣8️⃣  取得法人資料

In [ ]:
df = data.get('institutional_investors_trading_summary:投信買賣超股數')
df.tail()

Daily usage: 18.4 / 500 MB - institutional_investors_trading_summary:投信買賣超股數


symbol,2302,2303,2311,2325,2329,2330,2337,2338,2340,2342,...,6573,6756,8016,8028,8081,8110,8131,8150,8261,8271
date,,,,,,,,,,,,,,,,,,,,,
2020-12-25,0.0,1743000.0,NaN,NaN,0.0,-420000.0,-685000.0,0.0,0.0,250000.0,...,0.0,0.0,8000.0,-122000.0,0.0,0.0,0.0,151000.0,0.0,0.0
2020-12-28,0.0,5804000.0,NaN,NaN,0.0,26000.0,816000.0,0.0,0.0,0.0,...,0.0,0.0,64000.0,-200000.0,0.0,0.0,0.0,-142000.0,0.0,0.0
2020-12-29,0.0,2141930.0,NaN,NaN,0.0,-88180.0,1449000.0,0.0,0.0,100000.0,...,0.0,0.0,-18000.0,-213000.0,0.0,0.0,0.0,2000.0,0.0,0.0
2020-12-30,0.0,452000.0,NaN,NaN,0.0,257000.0,3507000.0,0.0,0.0,0.0,...,0.0,0.0,27000.0,-104000.0,0.0,0.0,0.0,-90000.0,0.0,0.0
2020-12-31,0.0,263000.0,NaN,NaN,0.0,419000.0,396000.0,0.0,0.0,0.0,...,0.0,0.0,-1000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#3.4 SQL資料庫

### 2️⃣9️⃣ 匯入套件及連線資料庫

In [ ]:
import sqlite3
conn = sqlite3.connect('stock.db')

### 3️⃣0️⃣設定資料庫結構

In [ ]:
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS 日頻資料 (
    sno INTEGER PRIMARY KEY AUTOINCREMENT,
    Stock_Id TEXT,
    Date DATE,
    Open FLOAT,
    High FLOAT,
    Low FLOAT,
    Close FLOAT,
    Adj_Close FLOAT,
    Volume INTEGER
);
''')
conn.commit()

### 3️⃣1️⃣ 傳入資料到資料庫

In [ ]:
df = yf.download('2330.TW',start='2023-08-01', auto_adjust=False, multi_level_index=False)
df = df.reset_index()
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
df.insert(0,'Stock_id','2330')

df.to_sql('日頻資料',conn,if_exists='append',index=False)

[*********************100%***********************]  1 of 1 completed


397

### 3️⃣2️⃣ 查詢表格資料

In [ ]:
def table_info(table_name):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    column_names = [column[1] for column in columns]
    print(f"資料庫表 '{table_name}' 的欄位名稱：", column_names)
    all_data = conn.execute(f'SELECT * FROM {table_name}')
    for i in all_data.fetchall():
        print(i)

# 查詢表格資料
table_info("日頻資料")

資料庫表 '日頻資料' 的欄位名稱： ['sno', 'Stock_Id', 'Date', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
(1, '2330', '2023-08-01', 565.0, 568.0, 564.0, 567.0, 549.020263671875, 16259643)
(2, '2330', '2023-08-02', 567.0, 569.0, 558.0, 561.0, 543.2105102539062, 25583234)
(3, '2330', '2023-08-04', 556.0, 560.0, 552.0, 554.0, 536.4324340820312, 26279173)
(4, '2330', '2023-08-07', 558.0, 561.0, 556.0, 558.0, 540.3056640625, 14369551)
(5, '2330', '2023-08-08', 558.0, 558.0, 551.0, 552.0, 534.495849609375, 19010690)
(6, '2330', '2023-08-09', 550.0, 557.0, 550.0, 554.0, 536.4324340820312, 14389090)
(7, '2330', '2023-08-10', 552.0, 554.0, 550.0, 551.0, 533.527587890625, 17892615)
(8, '2330', '2023-08-11', 556.0, 558.0, 546.0, 546.0, 528.6861572265625, 17608427)
(9, '2330', '2023-08-14', 540.0, 543.0, 537.0, 541.0, 523.8446044921875, 20939030)
(10, '2330', '2023-08-15', 543.0, 545.0, 540.0, 542.0, 524.81298828125, 14066307)
(11, '2330', '2023-08-16', 541.0, 543.0, 536.0, 542.0, 524.81298828125, 282

### 3️⃣3️⃣ 新增資料

In [ ]:
def insert_data(stock_id, start):
  # 下載資料
  df = yf.download(f'{stock_id}.TW',start=start, auto_adjust=False, multi_level_index=False)
  df = df.reset_index()
  df.rename(columns={"Adj Close": "Adj_Close"}, inplace=True)
  df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
  df.insert(0,'Stock_Id',stock_id)

  # 新增資料表
  df.to_sql('日頻資料',conn,if_exists='append',index=False)

# 新增 2317 資料
insert_data(stock_id=2317, start='2023-08-01')

[*********************100%***********************]  1 of 1 completed


### 3️⃣4️⃣ 從資料庫取得資料

In [ ]:
query = ("SELECT Stock_id, Date, Close "
         "FROM 日頻資料 "
         "WHERE Date < '2023-08-15' AND Stock_id = '2317'")
df = pd.read_sql(query, conn, parse_dates=['Date'])
df.tail()

,Stock_Id,Date,Close
4,2317,2023-08-08,110.5
5,2317,2023-08-09,110.5
6,2317,2023-08-10,110.0
7,2317,2023-08-11,108.5
8,2317,2023-08-14,110.0


### 3️⃣5️⃣ 修改並關閉資料庫

In [ ]:
conn.commit()
conn.close()